### Find SS

In [13]:
import numpy as np
from scipy.optimize import root
from matplotlib import pyplot as plt
import time

beta = .96
delta = 0.06
A = 1
alpha = .36
SStol = 1.E-10
xi = .4
lbar = .95
pi_11 = .9 
pi_12 = 1 - pi_11
pi_21 = (lbar - (lbar * pi_11))/(1-lbar)
pi_22 = ((1-lbar) - (lbar * pi_12))/(1-lbar)
PI = np.array([[pi_11, pi_12], [pi_21, pi_22]])
print(PI)


[[ 0.9  0.1]
 [ 1.9 -0.9]]


In [14]:
def modeldef(K, beta, L, A, alpha, delta, PI):
    '''
        Defines the model
        Inputs:
            v - vector of solutions 
            beta - discount rate
            sigma - rate of relative risk aversion
            nvec - how much each population works 
            L - aggregate labor 
            A - technology
            alpha - capital share
            delta - depreciation
        Return:
            list of functions
    '''
    r = ( ( alpha * A * ( (L / K))**(1-alpha) ))
    w = (  (1-alpha) * A * ( ( K / L)**(alpha) ) )

